In [10]:
from copy import copy
import astropy.units as u
from gammapy.utils.fits import HDULocation
from gammapy.data import DataStore, GTI
from gammapy.data.data_store import REQUIRED_IRFS, ALL_HDUS, ALL_IRFS
from ctao_data_model import IRFGroup, IRFGroups, VODFObservation

Here we create a function to extract  from a `HDUIndexTable` a GADF observation as a `VODFObservation`. 
The `IRFGroup` is created from the list of IRFs locations to use the lazy loading mechanism.


In [2]:
def get_obs(hdu_table, obs_id, required_irf="full-enclosure", require_events=True):
    """Access a given `~gammapy.data.Observation` from a HDU index table.

    Parameters
    ----------
    hdu_table : `~gammapy.data.`
        Input index table
    obs_id : int
        Observation ID.
    required_irf : list of str or str, optional
        The list can include the following options:

        * `"events"` : Events
        * `"gti"` :  Good time intervals
        * `"aeff"` : Effective area
        * `"bkg"` : Background
        * `"edisp"` : Energy dispersion
        * `"psf"` : Point Spread Function
        * `"rad_max"` : Maximal radius

        Alternatively single string can be used as shortcut:

        * `"full-enclosure"` : includes `["events", "gti", "aeff", "edisp", "psf", "bkg"]`
        * `"point-like"` : includes `["events", "gti", "aeff", "edisp"]`

        Default is `"full-enclosure"`.
    require_events : bool, optional
        Require events and gti table or not. Default is True.

    Returns
    -------
    VODFobservation : `~cato_data_model.VODFObservation`
        Observation container.

    """
    if obs_id not in hdu_table["OBS_ID"]:
        raise ValueError(f"OBS_ID = {obs_id} not in HDU index table.")

    kwargs = {"obs_id": int(obs_id)}

    # check for the "short forms"
    if isinstance(required_irf, str):
        required_irf = REQUIRED_IRFS[required_irf]

    if not set(required_irf).issubset(ALL_IRFS):
        difference = set(required_irf).difference(ALL_IRFS)
        raise ValueError(
            f"{difference} is not a valid hdu key. Choose from: {ALL_IRFS}"
        )

    if require_events:
        required_hdus = {"events", "gti"}.union(required_irf)
    else:
        required_hdus = required_irf

    missing_hdus = []
    kwargs_irf = {}
    for hdu in ALL_HDUS:
        hdu_location = hdu_table.hdu_location(
            obs_id=obs_id,
            hdu_type=hdu,
            warn_missing=False,
        )
        if hdu_location is not None:
            if hdu in ALL_IRFS:
                kwargs_irf[hdu] = hdu_location
            else:
                kwargs[hdu] = hdu_location
        elif hdu in required_hdus:
            missing_hdus.append(hdu)


    
    if len(missing_hdus) > 0:
        raise MissingRequiredHDU(
            f"Required HDUs {missing_hdus} not found in observation {obs_id}"
        )

    # TODO: right now, gammapy doesn't support using the pointing table of GADF
    # so we always pass the events location here to be read into a FixedPointingInfo
    if "events" in kwargs:
        pointing_location = copy(kwargs["events"])
        pointing_location.hdu_class = "pointing"
        kwargs["pointing"] = pointing_location

    kwargs["irf_groups"] = IRFGroups([IRFGroup(**kwargs_irf)])
    
    return VODFObservation(**kwargs)

In [11]:
datastore = DataStore.from_dir("$GAMMAPY_DATA/hess-dl3-dr1/")

In [12]:
grp = IRFGroup.from_hdu_index_table(datastore.hdu_table, 23523)

{'gti': <gammapy.utils.fits.HDULocation object at 0x1509a13f0>, 'aeff': <gammapy.utils.fits.HDULocation object at 0x15092d930>, 'edisp': <gammapy.utils.fits.HDULocation object at 0x15092ee00>, 'psf': <gammapy.utils.fits.HDULocation object at 0x15092ed40>, 'bkg': <gammapy.utils.fits.HDULocation object at 0x15092dc90>}


In [14]:
time_range1 = GTI.from_time_intervals([grp.gti.time_start + [0, 5]*u.min])
grp.gti = time_range1

In [15]:
grp.gti

In [ ]:
isinstance(obs.edisp, HDULocation)

In [ ]:
grp.gti